# ISA on scclevr with MyBigResNet as encoder

Using the common Evalution Framework, I like to compare it to the benchmark ISA!

This is a bigger ResNet then the "MyResNet" but also does not use bottleneck tecinque.

In [1]:
import torch
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import matplotlib as mlp
from mpl_toolkits.axes_grid1 import make_axes_locatable

import json, yaml, os
os.sys.path.append('./../../code')

from plotting import plot_kslots, plot_kslots_iters
from data import make_batch
from model import InvariantSlotAttention

from matplotlib.patches import Circle
import json

# Set numpy seed for test set sampling 
torch_seed = 24082023
torch.manual_seed( torch_seed )

import random
random.seed(torch_seed)

%load_ext autoreload
%autoreload 2

In [2]:
device = 'cpu'

In [3]:
cID_prev = 'isa-scclevr-EncStudy-myBigResNet-longTraining'
with open(f'./../../code/configs/{cID_prev}.yaml') as f:
    cd = yaml.safe_load(f)

hps = cd['hps']
hps['device'] = device

In [4]:
m = InvariantSlotAttention(**hps)

Using MyBigResNet to encode data.


In [5]:
f = open(f'./../../code/models/{cID_prev}/loss.json')
# returns JSON object as a dictionary
losses = json.load(f)

FileNotFoundError: [Errno 2] No such file or directory: './../../code/models/isa-scclevr-EncStudy-myBigResNet-longTraining/loss.json'

In [ ]:
for k,v in losses.items():
    plt.plot(v,label=k)
    print("loss ", k, " min: ", np.min(v), " at ", np.argmin(v))
plt.xlabel('Iters')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
lastIter = 15000
weightPath = f'./../../code/models/{cID_prev}/m_{lastIter}.pt'
print(f'Starting from an earlier training',lastIter)

m.load_state_dict(torch.load(weightPath,map_location=device))

### Evaluate the model

Use the common evaluation framework

In [ ]:
import scclevr
os.sys.path.append('./../../../evaluation')
from dataset import TestDataset
import model_performance as ev
%load_ext autoreload
%autoreload 2

In [ ]:
ds = TestDataset()
X = torch.FloatTensor(ds.event_images).to(device)

In [ ]:
with torch.no_grad():
    queries, att, Y_pred = m(X) 

In [ ]:
mp = ev.Model_performance(ds, att)
mp.hungarian_matching()

In [ ]:
for i in range(5):
    mp.plot_instance_i(i)

In [ ]:
loss = mp.KL_loss()
np.log10(loss)

In [ ]:
n, bins = mp.loss_hist()

In [ ]:
#with open('MyBigResNet.npy', 'wb') as f:
#    np.save(f, n)
#    np.save(f, bins)

Mhhhh .... Best network until now and still not as good as Florians...why?



In [ ]:
mp.Pearson_correlation_coefficients()

In [ ]:
mp.KDE_hist()

### Plot loss as a function to of the distance to the border!

In [ ]:
df = mp._make_dataframe()

In [ ]:
df

In [ ]:
df.shape

In [ ]:
overlap_edge_obj1 = [True if (abs(df["x1"][i])+df["r1"][i]>0.5) or (abs(df["y1"][i])+df["r1"][i]>0.5) else False for i in range(df.shape[0])]
overlap_edge_obj2 = [True if (abs(df["x2"][i])+df["r2"][i]>0.5) or (abs(df["y2"][i])+df["r2"][i]>0.5) else False for i in range(df.shape[0])]

In [ ]:
overlap_edge_any = [overlap_edge_obj1 or overlap_edge_obj2]

In [ ]:
len(overlap_edge_any[0])

In [ ]:
overlap_edge_both = [overlap_edge_obj1 and overlap_edge_obj2]

In [ ]:
overlap_edge = [2 if (overlap_edge_obj1[i] and overlap_edge_obj2[i]) else (1 if overlap_edge_obj1[i] or overlap_edge_obj2[i] else 0) for i in range(df.shape[0])]

In [ ]:
df["overlap_edge"] = overlap_edge

In [ ]:
import seaborn as sns

In [ ]:
df["overlap_edge"].value_counts(normalize=True)

In [ ]:
sns.kdeplot(data=df, x="log_loss", hue='overlap_edge', fill=True)

In [ ]:
overlap_edge_value =abs(df["x1"][overlap_edge_obj1])+df["r1"][overlap_edge_obj1] + abs(df["y1"][overlap_edge_obj1])+df["r1"][overlap_edge_obj1] + abs(df["x2"][overlap_edge_obj2])+df["r2"][overlap_edge_obj2] + abs(df["y2"][overlap_edge_obj2])+df["r2"][overlap_edge_obj2]



In [ ]:
overlap_edge_value.shape

In [ ]:
o_x1 = [abs(df["x1"][i])+df["r1"][i] if (abs(df["x1"][i])+df["r1"][i]>0.5) else 0 for i in range(df.shape[0])]
o_y1 = [abs(df["y1"][i])+df["r1"][i] if (abs(df["y1"][i])+df["r1"][i]>0.5) else 0 for i in range(df.shape[0])]
o_x2 = [abs(df["x2"][i])+df["r2"][i] if (abs(df["x2"][i])+df["r2"][i]>0.5) else 0 for i in range(df.shape[0])]
o_y2 = [abs(df["y2"][i])+df["r2"][i] if (abs(df["y2"][i])+df["r2"][i]>0.5) else 0 for i in range(df.shape[0])]

In [ ]:
overlap_edge_value = np.array(o_x1) +np.array(o_y1)+np.array(o_x2)+np.array(o_y2)

In [ ]:
overlap_edge_value.shape

In [ ]:
df["overlap_edge_value"] = overlap_edge_value

In [ ]:
sns.histplot(data=df, x="log_loss", y="overlap_edge_value", cbar=True, )

In [ ]:
df["log_loss"]

In [ ]:
from matplotlib.colors import LogNorm

In [ ]:
norm = LogNorm(vmin=10**(-8), vmax=10**(-1))

In [ ]:
df["log_loss"].describe()

In [ ]:
fig = plt.figure(figsize=(12, 4))
fig.add_subplot(121)
sns.histplot(data=df, x='x1', y='y1', bins=50, cbar=True)
plt.title("(x,y) histogram of hits")
fig.add_subplot(122)
plt.title("(x,y) histogram of hits weighted with -log_loss")
sns.histplot(data=df, x='x1', y='y1', weights=-df["log_loss"], bins=50, cbar=True)

## Compare with smaler ResNet 

In [ ]:
with open('./loss_histograms/MyResNet.npy', 'rb') as f:
    n1 = np.load(f)
    bins1 = np.load(f)

In [ ]:
ev.compare_histograms(np.array([n, n1]), np.array([bins, bins1]), ['Big ResNet: -3.615', 'Small ResNet: -3.42'], xlabel='log KL loss')